In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
train_data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/DataSets/Kaggle/cis-Fraud/train_transaction.csv',index_col='TransactionID')
train_ident = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/DataSets/Kaggle/cis-Fraud/train_identity.csv',index_col='TransactionID')

test_ident = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/DataSets/Kaggle/cis-Fraud/test_identity.csv',index_col='TransactionID')
test_data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/DataSets/Kaggle/cis-Fraud/test_transaction.csv',index_col='TransactionID')

train_data = train_data.merge(train_ident, how='left', left_index=True, right_index=True)
test_data = test_data.merge(test_ident, how='left', left_index=True, right_index=True)



In [ ]:
one_value_cols = [col for col in train_data.columns if train_data[col].nunique() <= 1] 

many_null_cols = [col for col in train_data.columns if train_data[col].isnull().sum() / train_data.shape[0] > 0.9] 

big_top_value_cols = [col for col in train_data.columns if train_data[col].value_counts(dropna=False, normalize=True).values[0] > 0.9] 

cols_to_drop = list(set(many_null_cols + big_top_value_cols + one_value_cols)) 

cols_to_drop.remove('isFraud') 

train_data = train_data.drop(cols_to_drop, axis=1)
test_data = test_data.drop(cols_to_drop, axis=1)




In [ ]:
import numpy as np
import datetime
from sklearn.preprocessing import LabelEncoder


big_data=pd.concat([train_data,test_data])

big_data["ProductCD"] = big_data["ProductCD"].map({"W":1, "H":2, "C" : 3 , "S":4, "R":5})
big_data["card4"] = big_data["card4"].map({"discover":1, "mastercard":2, "visa" : 3 , "american express":4})
big_data["card6"] = big_data["card6"].map({"debit":1, "credit":2, "debit or credit" : 3 , "charge card":4})
big_data["M2"] = big_data["M2"].map({"T":1, "F":2})
big_data["M3"] = big_data["M3"].map({"T":1, "F":2})
big_data["M3"] = big_data["M3"].map({"T":1, "F":2})
big_data["M4"] = big_data["M4"].map({"M0":1, "M1":2, "M2" : 3})
big_data["M5"] = big_data["M5"].map({"T":1, "F":2})
big_data["M6"] = big_data["M6"].map({"T":1, "F":2})
big_data["M7"] = big_data["M7"].map({"T":1, "F":2})
big_data["M8"] = big_data["M8"].map({"T":1, "F":2})
big_data["M9"] = big_data["M9"].map({"T":1, "F":2})

big_data["id_12"] = big_data["id_12"].map({"NotFound":1, "Found":2})
big_data["id_15"] = big_data["id_15"].map({"New":1, "Found":2, "Unknown" : 3})
big_data["id_36"] = big_data["id_36"].map({"T":1, "F":2})
big_data["id_37"] = big_data["id_37"].map({"T":1, "F":2})
big_data["id_38"] = big_data["id_38"].map({"T":1, "F":2})
big_data["DeviceType"] = big_data["DeviceType"].map({"mobile":1, "desktop":2})

big_data['id_33']=big_data['id_33'].astype(str)
big_data['id_33'] = big_data['id_33'].apply(lambda x: int(x.split('x')[0])*int(x.split('x')[1]) if x!='nan' else -999)
big_data['id_33']=big_data['id_33'].astype(int)

big_data['TransactionAmt_decimal'] = ((big_data['TransactionAmt'] - big_data['TransactionAmt'].astype(int)) * 1000).astype(int)

big_data['Transaction_day_of_week'] = np.floor((big_data['TransactionDT'] / (3600 * 24) - 1) % 7)
big_data['Transaction_hour'] = np.floor(big_data['TransactionDT'] / 3600) % 24

big_data['P_emaildomain']=big_data['P_emaildomain'].fillna('None')
big_data['onlyhostsP']=big_data['P_emaildomain'].apply(lambda x: x.split('.')[0] if '.' in x else x)
big_data['onlylocP']=big_data['P_emaildomain'].apply(lambda x: x.split('.')[1] if '.' in x else 'None')

big_data['R_emaildomain']=big_data['R_emaildomain'].fillna('None')
big_data['onlyhostsR']=big_data['R_emaildomain'].apply(lambda x: x.split('.')[0] if '.' in x else x)
big_data['onlylocR']=big_data['R_emaildomain'].apply(lambda x: x.split('.')[1] if '.' in x else 'None')

big_data['id_30']=big_data['id_30'].fillna('None')
big_data['id_30']=big_data['id_30'].apply(lambda x: x.split(' ')[0] if ' ' in x else x)

big_data['id_31']=big_data['id_31'].fillna('None')
big_data['id_31']=(big_data['id_31'].apply(lambda x: x.split(' ')[0] if ' ' in x else x)).apply(lambda x: x.split('/')[0] if '/' in x else x)

big_data['D9'] = (big_data['TransactionDT']%(3600*24)/3600//1)/24.0


big_data['card1_count_full'] = big_data['card1'].map(big_data['card1'].value_counts(dropna=False))

for feature in ['id_02__id_20', 'id_02__D8', 'D11__DeviceInfo', 'DeviceInfo__P_emaildomain', 'P_emaildomain__C2', 
                'card2__dist1', 'card1__card5', 'card2__id_20', 'card5__P_emaildomain', 'addr1__card1']:

    f1, f2 = feature.split('__')
    big_data[feature] = big_data[f1].astype(str) + '_' + big_data[f2].astype(str)

    le = LabelEncoder()
    le.fit(list(big_data[feature].astype(str).values))
    big_data[feature] = le.transform(list(big_data[feature].astype(str).values))
    
    
for feature in ['id_34', 'id_36']:
        big_data[feature + '_count_full'] = big_data[feature].map(big_data[feature].value_counts(dropna=False))
         
for feature in ['id_01', 'id_31', 'id_33', 'id_35', 'id_36']:
        big_data[feature + '_count_dist'] = big_data[feature].map(big_data[feature].value_counts(dropna=False))



big_data['DeviceInfo']=big_data['DeviceInfo'].fillna('None')
big_data['None?']=big_data['DeviceInfo'].apply(lambda x: 1 if 'None' in x else 0)
big_data['Windows?']=big_data['DeviceInfo'].apply(lambda x: 1 if 'Windows' in x else 0)
big_data['iOS?']=big_data['DeviceInfo'].apply(lambda x: 1 if 'iOS' in x else 0)
big_data['MacOS?']=big_data['DeviceInfo'].apply(lambda x: 1 if 'MacOS' in x else 0)
big_data['SM?']=big_data['DeviceInfo'].apply(lambda x: 1 if 'SM' in x else 0)
big_data['Trident?']=big_data['DeviceInfo'].apply(lambda x: 1 if 'Trident' in x else 0)
big_data['SAMSUNG?']=big_data['DeviceInfo'].apply(lambda x: 1 if 'SAMSUNG' in x else 0)
big_data['Moto?']=big_data['DeviceInfo'].apply(lambda x: 1 if 'Moto' in x else 0)
big_data['LG?']=big_data['DeviceInfo'].apply(lambda x: 1 if 'LG' in x else 0)
big_data['rv:?']=big_data['DeviceInfo'].apply(lambda x: 1 if 'rv:' in x else 0)
big_data['HUAWEI?']=big_data['DeviceInfo'].apply(lambda x: 1 if 'HUAWEI' in x else 0)
big_data['HTC?']=big_data['DeviceInfo'].apply(lambda x: 1 if 'HTC' in x else 0)
big_data['ALE?']=big_data['DeviceInfo'].apply(lambda x: 1 if 'ALE' in x else 0)
big_data['Lenovo?']=big_data['DeviceInfo'].apply(lambda x: 1 if 'Lenovo' in x else 0)
big_data['Blade?']=big_data['DeviceInfo'].apply(lambda x: 1 if 'Blade' in x else 0)
big_data['Pixel?']=big_data['DeviceInfo'].apply(lambda x: 1 if 'Pixel' in x else 0)
big_data['Ilium ?']=big_data['DeviceInfo'].apply(lambda x: 1 if 'Ilium ' in x else 0)
big_data['Hisense?']=big_data['DeviceInfo'].apply(lambda x: 1 if 'Hisense' in x else 0)
big_data['hi6210sft?']=big_data['DeviceInfo'].apply(lambda x: 1 if 'hi6210sft' in x else 0)


big_data['TransactionAmt_to_mean_card1'] = big_data['TransactionAmt'] / big_data.groupby(['card1'])['TransactionAmt'].transform('mean')
big_data['TransactionAmt_to_mean_card4'] = big_data['TransactionAmt'] / big_data.groupby(['card4'])['TransactionAmt'].transform('mean')
big_data['TransactionAmt_to_std_card1'] = big_data['TransactionAmt'] / big_data.groupby(['card1'])['TransactionAmt'].transform('std')
big_data['TransactionAmt_to_std_card4'] = big_data['TransactionAmt'] / big_data.groupby(['card4'])['TransactionAmt'].transform('std')

big_data['id_02_to_mean_card1'] = big_data['id_02'] / big_data.groupby(['card1'])['id_02'].transform('mean')
big_data['id_02_to_mean_card4'] = big_data['id_02'] / big_data.groupby(['card4'])['id_02'].transform('mean')
big_data['id_02_to_std_card1'] = big_data['id_02'] / big_data.groupby(['card1'])['id_02'].transform('std')
big_data['id_02_to_std_card4'] = big_data['id_02'] / big_data.groupby(['card4'])['id_02'].transform('std')

big_data['D15_to_mean_card1'] = big_data['D15'] / big_data.groupby(['card1'])['D15'].transform('mean')
big_data['D15_to_mean_card4'] = big_data['D15'] / big_data.groupby(['card4'])['D15'].transform('mean')
big_data['D15_to_std_card1'] = big_data['D15'] / big_data.groupby(['card1'])['D15'].transform('std')
big_data['D15_to_std_card4'] = big_data['D15'] / big_data.groupby(['card4'])['D15'].transform('std')


big_data['D15_to_mean_addr1'] = big_data['D15'] / big_data.groupby(['addr1'])['D15'].transform('mean')
big_data['D15_to_mean_card4'] = big_data['D15'] / big_data.groupby(['card4'])['D15'].transform('mean')
big_data['D15_to_std_addr1'] = big_data['D15'] / big_data.groupby(['addr1'])['D15'].transform('std')
big_data['D15_to_std_card4'] = big_data['D15'] / big_data.groupby(['card4'])['D15'].transform('std')

START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')
big_data['DT_M'] = big_data['TransactionDT'].apply(lambda x: (START_DATE + datetime.timedelta(seconds = x)))
big_data['DT_M'] = (big_data['DT_M'].dt.year-2017)*12 + big_data['DT_M'].dt.month 

big_data['TransactionAmt_check'] = np.where(big_data['TransactionAmt'].isin(big_data['TransactionAmt']), 1, 0)


#newFeatures---------------------------------------------
big_data['new_adrr2_corr<']=big_data['addr2'].apply(lambda x:1 if x<90 else 0)
big_data['new_adrr1_corr<']=big_data['addr1'].apply(lambda x:1 if x<536 else 0)
big_data['new_card2_corr<']=big_data['card2'].apply(lambda x:1 if x<599 else 0)
big_data['new_adrr2_corr>']=big_data['addr2'].apply(lambda x:1 if x>68 else 0)
big_data['new_adrr2_corr><']=big_data['addr2'].apply(lambda x:1 if x>68 and x<90 else 0)
big_data['new_card5_corr>']=big_data['card5'].apply(lambda x:1 if x>162 else 0)
big_data['new_Transaction_hour_corr>']=big_data['Transaction_hour'].apply(lambda x:1 if x>11 else 0)
#--------------------------------------------------------
cols_to_drop=['P_emaildomain','R_emaildomain','DeviceInfo']

big_data = big_data.drop(cols_to_drop, axis=1)


forencode=big_data.select_dtypes('object')

encoded=pd.get_dummies(forencode)

#drop needed col
for i in list(forencode):
    big_data = big_data.drop(i,axis = 1)
# Join the encoded df

big_data = big_data.join(encoded)

In [ ]:
train_data = big_data.iloc[:train_data.shape[0],]
test_data = big_data.iloc[train_data.shape[0]:,]

train_data=train_data.fillna(-999)
test_data=test_data.fillna(-999)

y=train_data['isFraud']
train_data=train_data.drop('isFraud',axis=1)
test_data=test_data.drop('isFraud',axis=1)

train_data=train_data.drop('TransactionDT',axis=1)
test_data=test_data.drop('TransactionDT',axis=1)



In [ ]:
from sklearn.model_selection import KFold, TimeSeriesSplit, StratifiedKFold
import lightgbm as lgb

n_fold = 10
#folds = KFold(n_splits=n_fold,shuffle=True,random_state=42)
folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)


sample_submission = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/DataSets/Kaggle/cis-Fraud/sample_submission.csv')

xgb_submission=sample_submission.copy()
xgb_submission['isFraud'] = 0
t=0

mean=[]

from sklearn.metrics import roc_auc_score
#for fold_n, (train_index, valid_index) in enumerate(folds.split(train_data)):
for train_index, valid_index in folds.split(train_data, y):
  #if t<3:
    lgbR=lgb.LGBMClassifier(
                objective='binary',
                boosting_type='gbdt',
                metric='auc',
                n_jobs=-1,
                learning_rate=0.064,
                num_leaves= 2**8,
                min_data_in_leaf=1000,
                max_depth=10,
                tree_learner='serial',
                colsample_bytree= 0.85,
                subsample_freq=1,
                subsample=0.85,
                n_estimators=2**9,
                max_bin=255,
                verbose=-1,
                seed= 47,
                early_stopping_rounds=100,
                reg_alpha=0.3,
                reg_lamdba=0.243
      )
    
    X_train_, X_valid = train_data.iloc[train_index], train_data.iloc[valid_index]
    y_train_, y_valid = y.iloc[train_index], y.iloc[valid_index]
    lgbR.fit(X_train_,y_train_,eval_metric='auc',eval_set=[(X_valid,y_valid)],verbose=50,early_stopping_rounds=100)
    del X_train_,y_train_
    pred=lgbR.predict_proba(test_data)[:,1]
    val=lgbR.predict_proba(X_valid)[:,1]
    del X_valid
    del lgbR
    print('ROC accuracy: {}'.format(roc_auc_score(y_valid, val)))
    mean.append(roc_auc_score(y_valid, val))
    del val,y_valid
    xgb_submission['isFraud'] = xgb_submission['isFraud']+pred/n_fold
    del pred
    t=t+1

m=0
for i in mean:
    m=m+i/len(mean)
print(m)

In [ ]:

from google.colab import files
xgb_submission.to_csv('predict2.csv',index=False) 
files.download('predict2.csv')

# popitka v NN #

In [ ]:
#neural nets test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    train_data, 
    y, 
    test_size=0.3,
    random_state=42, 
    shuffle=True)

X_train=X_train.values
y_train=y_train.values
X_test=X_test.values
y_test=y_test.values
test_data=test_data.values

In [ ]:
X_train = X_train.astype(int)
X_test = X_test.astype(int)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,LSTM,Lambda,Flatten
from keras.utils import np_utils

y_train_bin=np_utils.to_categorical(y_train)
y_test_bin=np_utils.to_categorical(y_test)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test= scaler.transform(X_test)

In [ ]:
#model
model=Sequential()
model.add(Dense(X_train.shape[1],input_dim=X_train.shape[1],kernel_initializer='normal',bias_initializer='TruncatedNormal',activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(X_train.shape[1]//2,kernel_initializer='normal',bias_initializer='TruncatedNormal',activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(2,kernel_initializer='normal',activation='softmax'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(X_train,y_train_bin,epochs=200,batch_size=500)

In [ ]:
from sklearn.metrics import roc_auc_score
val=model.predict_proba(X_test)[:,1]
print('ROC accuracy: {}'.format(roc_auc_score(y_test, val)))

In [ ]:
y_pred=model.predict(X_test)
y_pred =(y_pred>0.5)[:,1]*1
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
sample_submission = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/DataSets/Kaggle/cis-Fraud/sample_submission.csv')

xgb_submission=sample_submission.copy()
xgb_submission['isFraud'] = model.predict_proba(test_data)[:,1]


from google.colab import files
xgb_submission.to_csv('predict2.csv',index=False) 
files.download('predict2.csv')